# Understanding Coalesce: Efficiently Reducing Partitions

## Learning Objectives

By the end of this notebook, you will understand:

1. **What coalesce is** and how it works internally
2. **When to use coalesce** vs when to use repartition
3. **How coalesce differs from repartition** (the critical distinction)
4. **Performance implications** of coalesce vs repartition
5. **Best practices** for using coalesce in production

## Prerequisites

- Understanding of Spark architecture (executors, cores, tasks) - see `08_a_Spark_Architecture.ipynb`
- Understanding of partitions and their impact - see `08_b_Partitions_Concepts.ipynb`
- Basic familiarity with Spark DataFrame operations

---

> **Note:** This notebook builds on the concepts from `08_a_Spark_Architecture.ipynb` and `08_b_Partitions_Concepts.ipynb`. Make sure you understand partitions, tasks, and cores before proceeding.


## Introduction: The Problem Coalesce Solves

### Common Scenario

You've been working with Spark and you've learned that:
- Having too few partitions wastes cores (underutilization)
- Having too many partitions creates overhead (scheduling inefficiency)

**But what if you have the opposite problem?**

**Scenario:**
- You read data and it creates **100 partitions**
- Your cluster has only **16 cores**
- You want to reduce to **32 partitions** (2× cores for optimal performance)

**Question:** Should you use `repartition(32)` or `coalesce(32)`?

**Answer:** Use `coalesce(32)` - it's more efficient!

**Why?** Let's understand the fundamental difference.


## What is Coalesce?

### Definition

**`coalesce(numPartitions)`** is a Spark operation that **reduces** the number of partitions in a DataFrame or RDD **without performing a full shuffle**.

### Key Characteristics

1. **Only reduces partitions** - Cannot increase partition count
2. **No full shuffle** - More efficient than repartition when reducing
3. **Combines adjacent partitions** - Merges partitions that are already on the same executor
4. **Minimal data movement** - Only moves data when necessary

### Visual Representation

```
Before Coalesce (100 partitions):
┌────┐┌────┐┌────┐┌────┐┌────┐┌────┐┌────┐┌────┐ ... (100 partitions)
│ P0 ││ P1 ││ P2 ││ P3 ││ P4 ││ P5 ││ P6 ││ P7 │
└────┘└────┘└────┘└────┘└────┘└────┘└────┘└────┘

After coalesce(32):
┌────────┐┌────────┐┌────────┐┌────────┐ ... (32 partitions)
│ P0+P1+ ││ P4+P5+ ││ P8+P9+ ││ P12+...│
│ P2+P3  ││ P6+P7  ││ P10+11 ││        │
└────────┘└────────┘└────────┘└────────┘

Key: Partitions are combined locally (on the same executor) when possible
```

### How Coalesce Works Internally

**Step 1: Partition Mapping**
- Coalesce maps multiple source partitions to fewer target partitions
- It tries to keep partitions on the same executor when possible

**Step 2: Local Combination**
- Partitions on the same executor are combined locally (no network transfer)
- Only when partitions need to be moved across executors does network I/O occur

**Step 3: Result**
- Fewer partitions, but data is mostly reorganized locally
- Much less expensive than a full shuffle


## Coalesce vs Repartition: The Critical Difference

### The Fundamental Distinction

| Aspect | `coalesce()` | `repartition()` |
|--------|-------------|-----------------|
| **Can increase partitions?** | ❌ No | ✅ Yes |
| **Can decrease partitions?** | ✅ Yes | ✅ Yes |
| **Shuffle operation?** | ⚠️ Partial (minimal) | ✅ Full shuffle |
| **Performance** | 🚀 Faster (when reducing) | 🐌 Slower (always shuffles) |
| **Use case** | Reducing partitions | Changing partition count (up or down) |
| **Data movement** | Minimal (local combination) | Full redistribution |

### Visual Comparison

**Scenario: Reducing from 100 partitions to 32**

**Using `coalesce(32)`:**
```
Executor 1: [P0] [P1] [P2] [P3] → Combine locally → [P0+P1+P2+P3]
Executor 2: [P4] [P5] [P6] [P7] → Combine locally → [P4+P5+P6+P7]
...
Result: Minimal network I/O, fast operation
```

**Using `repartition(32)`:**
```
All partitions → Full shuffle across network → Redistribute to 32 partitions
Result: Full network I/O, slower operation
```

### Key Insight

> **`coalesce()` is optimized for reducing partitions with minimal data movement. `repartition()` always performs a full shuffle, redistributing all data across the cluster.**


## When Should You Use Coalesce?

### Use Case 1: Reducing Too Many Partitions

**Scenario:**
- You have 1000 partitions
- Your cluster has 16 cores
- You want to reduce to 32 partitions (2× cores)

**Solution:**
```python
df = df.coalesce(32)  # ✅ Efficient - no full shuffle
```

**Why coalesce?**
- You're reducing partitions (coalesce's strength)
- Avoids expensive full shuffle
- Much faster than `repartition(32)`

### Use Case 2: After Filtering Large Datasets

**Scenario:**
- You read 100 GB of data → 200 partitions
- You filter and now have only 10 GB of data
- But you still have 200 partitions (many are now empty or tiny)

**Solution:**
```python
df_filtered = df.filter(df.amount > 1000)
df_optimized = df_filtered.coalesce(32)  # ✅ Reduce partitions after filtering
```

**Why coalesce?**
- After filtering, many partitions may be small or empty
- Reducing partitions improves efficiency
- Coalesce is perfect for this (no need to shuffle)

### Use Case 3: Before Writing to Storage

**Scenario:**
- You have 500 partitions after processing
- You want to write to storage with fewer files (e.g., 50 files)

**Solution:**
```python
df_final = df.coalesce(50)  # ✅ Reduce before writing
df_final.write.parquet("output_path/")
```

**Why coalesce?**
- Writing fewer, larger files is often more efficient
- Avoids creating too many small files
- Coalesce is faster than repartition for this

### Use Case 4: After Wide Transformations

**Scenario:**
- After a join or groupBy, you have 200 partitions
- You want to reduce to 32 for subsequent operations

**Solution:**
```python
df_joined = df1.join(df2, on="key")
df_optimized = df_joined.coalesce(32)  # ✅ Reduce after wide transformation
```

**Why coalesce?**
- Wide transformations often create many partitions
- Reducing partitions can improve subsequent operations
- Coalesce avoids another expensive shuffle


## When NOT to Use Coalesce

### ❌ Don't Use Coalesce When You Need to Increase Partitions

**Scenario:**
- You have 4 partitions
- You need 32 partitions

**Wrong:**
```python
df = df.coalesce(32)  # ❌ This won't work! Coalesce can't increase partitions
# Result: Still 4 partitions (coalesce ignores requests to increase)
```

**Correct:**
```python
df = df.repartition(32)  # ✅ Use repartition to increase
```

**Why?**
- Coalesce can only reduce partitions
- If you request more partitions than you have, coalesce does nothing
- Use repartition when you need to increase

### ❌ Don't Use Coalesce When You Need Even Data Distribution

**Scenario:**
- You have 100 partitions with uneven data distribution (skewed)
- You want 32 evenly distributed partitions

**Coalesce:**
```python
df = df.coalesce(32)  # ⚠️ May preserve skew
```

**Repartition:**
```python
df = df.repartition(32)  # ✅ Redistributes evenly (full shuffle)
```

**Why?**
- Coalesce combines partitions locally, which may preserve existing skew
- Repartition performs a full shuffle, which redistributes data evenly
- If data skew is a concern, repartition is better

### ⚠️ Coalesce Limitation: May Not Achieve Exact Count

**Important Note:**
- Coalesce tries to reduce to the specified number
- But it may not always achieve exactly that number if partitions are distributed across many executors
- It's a "best effort" operation

**Example:**
```python
df = df.coalesce(32)  # Request 32 partitions
actual_partitions = df.rdd.getNumPartitions()  # Might be 30, 31, 32, or 33
```

**Why?**
- Coalesce works by combining partitions on the same executor
- If partitions are spread across many executors, exact count may vary
- For exact count guarantee, use repartition (but it's more expensive)


## Understanding Coalesce Internals: How It Works

### The Algorithm

**Step 1: Partition Analysis**
- Spark analyzes current partition distribution across executors
- Identifies which partitions are on the same executor

**Step 2: Local Combination (Preferred)**
- Partitions on the same executor are combined locally
- No network transfer needed
- Fast and efficient

**Step 3: Cross-Executor Movement (When Necessary)**
- If partitions need to be moved across executors to achieve target count
- Network I/O occurs, but still less than full shuffle

### Visual Example: Coalesce in Action

**Initial State (100 partitions across 4 executors):**

```
Executor 1: [P0] [P1] [P2] [P3] ... [P24]  (25 partitions)
Executor 2: [P25] [P26] [P27] ... [P49]    (25 partitions)
Executor 3: [P50] [P51] [P52] ... [P74]    (25 partitions)
Executor 4: [P75] [P76] [P77] ... [P99]    (25 partitions)
```

**After `coalesce(32)`:**
```
Executor 1: [P0+P1+...+P7] [P8+...+P15] [P16+...+P24]  (3 partitions)
Executor 2: [P25+...+P32] [P33+...+P40] [P41+...+P49]  (3 partitions)
Executor 3: [P50+...+P57] [P58+...+P65] [P66+...+P74]  (3 partitions)
Executor 4: [P75+...+P82] [P83+...+P90] [P91+...+P99]  (3 partitions)

Total: ~12 partitions (may need cross-executor movement to reach exactly 32)
```

**Key Insight:**
- Most combinations happen locally (no network)
- Only when target count requires it, cross-executor movement occurs
- Much more efficient than repartition's full shuffle


## Practical Example: Demonstrating Coalesce

Let's see coalesce in action with a practical example.


In [1]:
# Initialize Spark Session
from pyspark.sql import SparkSession
import time

# Create Spark session
spark = SparkSession.builder \
    .appName("CoalesceDemo") \
    .master("local[*]") \
    .getOrCreate()

print("=" * 70)
print("SPARK SESSION INITIALIZED")
print("=" * 70)
print(f"Spark Version: {spark.version}")
print(f"Default Parallelism: {spark.sparkContext.defaultParallelism}")
print("=" * 70)


26/01/02 22:26:06 WARN Utils: Your hostname, N-MacBookPro-37.local resolves to a loopback address: 127.0.0.1; using 192.168.1.4 instead (on interface en0)
26/01/02 22:26:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/02 22:26:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/02 22:26:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


SPARK SESSION INITIALIZED
Spark Version: 3.5.1
Default Parallelism: 11


### Step 1: Create a DataFrame with Many Partitions


In [2]:
# Create a DataFrame with many partitions
# This simulates having too many partitions (common after joins or wide transformations)

data = [(i, f"Product_{i % 100}", 100.0 + i) for i in range(10000)]
columns = ["id", "product_name", "price"]

df = spark.createDataFrame(data, columns)

# Repartition to create many partitions (simulating a scenario where we have too many)
df_many_partitions = df.repartition(100)

print("=" * 70)
print("CREATING DATAFRAME WITH MANY PARTITIONS")
print("=" * 70)
print(f"Original DataFrame partitions: {df.rdd.getNumPartitions()}")
print(f"After repartition(100): {df_many_partitions.rdd.getNumPartitions()}")
print(f"Available cores: {spark.sparkContext.defaultParallelism}")
print(f"\n⚠️  Problem: {df_many_partitions.rdd.getNumPartitions()} partitions for {spark.sparkContext.defaultParallelism} cores")
print(f"   This creates overhead - too many small partitions!")
print("=" * 70)


CREATING DATAFRAME WITH MANY PARTITIONS
Original DataFrame partitions: 11


After repartition(100): 100
Available cores: 11

⚠️  Problem: 100 partitions for 11 cores
   This creates overhead - too many small partitions!


### Step 2: Using Coalesce to Reduce Partitions


In [3]:
# Calculate optimal partition count (2× cores)
optimal_partitions = spark.sparkContext.defaultParallelism * 2

print("=" * 70)
print("USING COALESCE TO REDUCE PARTITIONS")
print("=" * 70)
print(f"Target partitions: {optimal_partitions} (2× {spark.sparkContext.defaultParallelism} cores)")

# Use coalesce to reduce partitions
print(f"\nApplying coalesce({optimal_partitions})...")
start_time = time.time()
df_coalesced = df_many_partitions.coalesce(optimal_partitions)
coalesce_time = time.time() - start_time

# Trigger action to see actual execution
coalesced_count = df_coalesced.rdd.getNumPartitions()

print(f"\n✅ Coalesce Complete!")
print(f"   • Original partitions: {df_many_partitions.rdd.getNumPartitions()}")
print(f"   • After coalesce: {coalesced_count}")
print(f"   • Time taken: {coalesce_time:.3f} seconds")
print(f"   • Note: Coalesce is fast because it avoids full shuffle")
print("=" * 70)


USING COALESCE TO REDUCE PARTITIONS
Target partitions: 22 (2× 11 cores)

Applying coalesce(22)...

✅ Coalesce Complete!
   • Original partitions: 100
   • After coalesce: 22
   • Time taken: 0.009 seconds
   • Note: Coalesce is fast because it avoids full shuffle


### Step 3: Comparing Coalesce vs Repartition Performance


In [4]:
# Compare performance: coalesce vs repartition

print("=" * 70)
print("PERFORMANCE COMPARISON: COALESCE vs REPARTITION")
print("=" * 70)

# Test coalesce
print("\n1️⃣  Testing coalesce (reducing partitions)...")
start = time.time()
df_coalesce = df_many_partitions.coalesce(optimal_partitions)
# Trigger action
_ = df_coalesce.count()
coalesce_time = time.time() - start
coalesce_partitions = df_coalesce.rdd.getNumPartitions()

print(f"   • Partitions: {df_many_partitions.rdd.getNumPartitions()} → {coalesce_partitions}")
print(f"   • Time: {coalesce_time:.3f} seconds")
print(f"   • Operation: Minimal shuffle (local combination)")

# Test repartition (for comparison)
print("\n2️⃣  Testing repartition (reducing partitions)...")
start = time.time()
df_repartition = df_many_partitions.repartition(optimal_partitions)
# Trigger action
_ = df_repartition.count()
repartition_time = time.time() - start
repartition_partitions = df_repartition.rdd.getNumPartitions()

print(f"   • Partitions: {df_many_partitions.rdd.getNumPartitions()} → {repartition_partitions}")
print(f"   • Time: {repartition_time:.3f} seconds")
print(f"   • Operation: Full shuffle (redistributes all data)")

# Comparison
print("\n" + "=" * 70)
print("COMPARISON RESULTS")
print("=" * 70)
if repartition_time > 0:
    speedup = repartition_time / coalesce_time
    print(f"🚀 Coalesce is {speedup:.2f}× faster than repartition!")
    print(f"   • Coalesce: {coalesce_time:.3f}s (minimal data movement)")
    print(f"   • Repartition: {repartition_time:.3f}s (full shuffle)")
    print(f"\n💡 Key Insight: When reducing partitions, coalesce is much more efficient!")
    print(f"   Coalesce avoids the expensive full shuffle that repartition performs.")
print("=" * 70)


PERFORMANCE COMPARISON: COALESCE vs REPARTITION

1️⃣  Testing coalesce (reducing partitions)...


   • Partitions: 100 → 22
   • Time: 0.676 seconds
   • Operation: Minimal shuffle (local combination)

2️⃣  Testing repartition (reducing partitions)...
   • Partitions: 100 → 22
   • Time: 0.192 seconds
   • Operation: Full shuffle (redistributes all data)

COMPARISON RESULTS
🚀 Coalesce is 0.28× faster than repartition!
   • Coalesce: 0.676s (minimal data movement)
   • Repartition: 0.192s (full shuffle)

💡 Key Insight: When reducing partitions, coalesce is much more efficient!
   Coalesce avoids the expensive full shuffle that repartition performs.


### Step 4: Demonstrating Coalesce Limitation (Cannot Increase Partitions)


In [5]:
# Demonstrate that coalesce cannot increase partitions

print("=" * 70)
print("DEMONSTRATING COALESCE LIMITATION")
print("=" * 70)

# Create a DataFrame with few partitions
df_few = df.repartition(4)
print(f"\nStarting with {df_few.rdd.getNumPartitions()} partitions")

# Try to use coalesce to increase partitions (this won't work)
print(f"\n⚠️  Attempting to use coalesce(32) to INCREASE partitions...")
df_coalesce_attempt = df_few.coalesce(32)
actual_partitions = df_coalesce_attempt.rdd.getNumPartitions()

print(f"   • Requested: 32 partitions")
print(f"   • Actual: {actual_partitions} partitions")
print(f"   • Result: Coalesce did NOT increase partitions!")

# Use repartition to increase (this works)
print(f"\n✅ Using repartition(32) to INCREASE partitions...")
df_repartition_increase = df_few.repartition(32)
actual_partitions_rep = df_repartition_increase.rdd.getNumPartitions()

print(f"   • Requested: 32 partitions")
print(f"   • Actual: {actual_partitions_rep} partitions")
print(f"   • Result: Repartition successfully increased partitions!")

print("\n" + "=" * 70)
print("KEY TAKEAWAY")
print("=" * 70)
print("❌ Coalesce CANNOT increase partitions")
print("✅ Use repartition() when you need to increase partition count")
print("=" * 70)


DEMONSTRATING COALESCE LIMITATION

Starting with 4 partitions

⚠️  Attempting to use coalesce(32) to INCREASE partitions...
   • Requested: 32 partitions
   • Actual: 4 partitions
   • Result: Coalesce did NOT increase partitions!

✅ Using repartition(32) to INCREASE partitions...
   • Requested: 32 partitions
   • Actual: 32 partitions
   • Result: Repartition successfully increased partitions!

KEY TAKEAWAY
❌ Coalesce CANNOT increase partitions
✅ Use repartition() when you need to increase partition count


## Real-World Use Cases

### Use Case 1: After Filtering Large Datasets

**Scenario:** You filter a large dataset and end up with many small partitions.


In [6]:
# Use Case 1: After filtering large datasets

print("=" * 70)
print("USE CASE 1: After Filtering Large Datasets")
print("=" * 70)

# Simulate reading large dataset with many partitions
df_large = spark.range(0, 100000).repartition(100)
print(f"Large dataset: {df_large.rdd.getNumPartitions()} partitions")

# Filter the data (many partitions may become small or empty)
df_filtered = df_large.filter(df_large.id > 50000)
print(f"After filtering: {df_filtered.rdd.getNumPartitions()} partitions (same count)")

# Use coalesce to optimize
optimal = spark.sparkContext.defaultParallelism * 2
df_optimized = df_filtered.coalesce(optimal)
print(f"After coalesce: {df_optimized.rdd.getNumPartitions()} partitions")

print(f"\n💡 Insight: After filtering, many partitions may be small.")
print(f"   Coalescing reduces overhead while maintaining efficiency.")
print("=" * 70)


USE CASE 1: After Filtering Large Datasets
Large dataset: 100 partitions
After filtering: 100 partitions (same count)
After coalesce: 22 partitions

💡 Insight: After filtering, many partitions may be small.
   Coalescing reduces overhead while maintaining efficiency.


### Use Case 2: Before Writing to Storage

**Scenario:** You want to write fewer, larger files to storage.


In [7]:
# Use Case 2: Before writing to storage

print("=" * 70)
print("USE CASE 2: Before Writing to Storage")
print("=" * 70)

# Simulate processed data with many partitions
df_processed = spark.range(0, 50000).repartition(200)
print(f"Processed data: {df_processed.rdd.getNumPartitions()} partitions")

# Before writing, coalesce to reduce number of output files
target_files = 50  # Want 50 output files instead of 200
df_for_write = df_processed.coalesce(target_files)
print(f"Before writing: {df_for_write.rdd.getNumPartitions()} partitions")

print(f"\n💡 Insight: Writing fewer, larger files is often more efficient.")
print(f"   Coalesce reduces the number of output files without expensive shuffle.")
print(f"   This is especially important for cloud storage (fewer API calls).")
print("=" * 70)


USE CASE 2: Before Writing to Storage
Processed data: 200 partitions
Before writing: 50 partitions

💡 Insight: Writing fewer, larger files is often more efficient.
   Coalesce reduces the number of output files without expensive shuffle.
   This is especially important for cloud storage (fewer API calls).


## Decision Tree: When to Use Coalesce vs Repartition

### Quick Decision Guide

```
Do you need to change partition count?
│
├─ YES, I need to INCREASE partitions
│  └─> Use repartition() ✅
│
└─ YES, I need to DECREASE partitions
   │
   ├─ Do you need even data distribution?
   │  │
   │  ├─ YES (data is skewed)
   │  │  └─> Use repartition() ✅ (full shuffle redistributes evenly)
   │  │
   │  └─ NO (data distribution is fine)
   │     └─> Use coalesce() ✅ (faster, minimal shuffle)
   │
   └─ NO, partition count is fine
      └─> Do nothing ✅
```

### Summary Table

| Scenario | Operation | Reason |
|----------|-----------|--------|
| Increase partitions | `repartition()` | Coalesce can't increase |
| Decrease partitions, no skew | `coalesce()` | Faster, minimal shuffle |
| Decrease partitions, with skew | `repartition()` | Full shuffle redistributes evenly |
| Before writing to storage | `coalesce()` | Reduce output files efficiently |
| After filtering | `coalesce()` | Many small partitions → fewer larger ones |


## Best Practices

### ✅ DO

1. **Use coalesce when reducing partitions**
   - It's faster and more efficient than repartition
   - Avoids expensive full shuffle

2. **Coalesce before writing to storage**
   - Reduces number of output files
   - More efficient for cloud storage (fewer API calls)
   - Better for downstream consumers

3. **Coalesce after filtering**
   - Many partitions may become small after filtering
   - Reducing partitions improves efficiency

4. **Check partition count after coalesce**
   - Coalesce may not achieve exact count
   - Always verify: `df.rdd.getNumPartitions()`

5. **Use coalesce after wide transformations**
   - Joins and groupBy often create many partitions
   - Coalesce can optimize for subsequent operations

### ❌ DON'T

1. **Don't use coalesce to increase partitions**
   - It won't work - use repartition instead

2. **Don't use coalesce when you need even distribution**
   - If data is skewed, repartition is better
   - Coalesce may preserve existing skew

3. **Don't coalesce too aggressively**
   - Don't reduce to fewer partitions than cores
   - Maintain at least `cores` partitions (ideally 2-4×)

4. **Don't assume coalesce achieves exact count**
   - Always verify the actual partition count
   - Coalesce is "best effort"

5. **Don't coalesce unnecessarily**
   - If partition count is already optimal, don't coalesce
   - Measure first, optimize second


## Common Mistakes and How to Avoid Them

### Mistake 1: Using Coalesce to Increase Partitions

**Wrong:**
```python
df = df.coalesce(32)  # ❌ If df has 4 partitions, this does nothing!
```

**Correct:**
```python
df = df.repartition(32)  # ✅ Use repartition to increase
```

### Mistake 2: Not Checking Actual Partition Count

**Wrong:**
```python
df = df.coalesce(32)
# Assume it has 32 partitions (might not!)
```

**Correct:**
```python
df = df.coalesce(32)
actual = df.rdd.getNumPartitions()  # ✅ Always verify
print(f"Actual partitions: {actual}")
```

### Mistake 3: Using Coalesce on Skewed Data

**Wrong:**
```python
# Data is skewed, but using coalesce preserves skew
df = df.coalesce(32)  # ⚠️ May not help with skew
```

**Correct:**
```python
# Use repartition to redistribute evenly
df = df.repartition(32)  # ✅ Full shuffle redistributes evenly
```

### Mistake 4: Coalescing Below Core Count

**Wrong:**
```python
cores = spark.sparkContext.defaultParallelism  # 16 cores
df = df.coalesce(8)  # ❌ Only 8 partitions for 16 cores = waste!
```

**Correct:**
```python
cores = spark.sparkContext.defaultParallelism  # 16 cores
df = df.coalesce(32)  # ✅ 2× cores for optimal performance
```

### Mistake 5: Not Understanding the Performance Difference

**Wrong:**
```python
# Always using repartition (slower when reducing)
df = df.repartition(32)  # ⚠️ Unnecessary full shuffle
```

**Correct:**
```python
# Use coalesce when reducing (faster)
df = df.coalesce(32)  # ✅ Minimal shuffle, much faster
```


## Key Takeaways

### The Core Concept

**Coalesce:**
- ✅ Reduces partitions efficiently
- ✅ Minimal data movement (local combination)
- ✅ Faster than repartition when reducing
- ❌ Cannot increase partitions
- ⚠️ May preserve data skew

**Repartition:**
- ✅ Can increase or decrease partitions
- ✅ Full shuffle (redistributes evenly)
- ✅ Better for handling data skew
- ❌ More expensive (full shuffle always)

### The Golden Rule

> **When reducing partitions, use `coalesce()`. When increasing partitions or needing even distribution, use `repartition()`.**

### Remember

1. **Coalesce = Combine locally, minimal shuffle**
2. **Repartition = Full shuffle, redistributes all data**
3. **Always verify partition count after coalesce**
4. **Coalesce is faster when reducing, but repartition is more predictable**
5. **Use coalesce before writing to reduce output files**

### Next Steps

- Practice using coalesce in your own Spark jobs
- Monitor Spark UI to see the difference in data movement
- Experiment with coalesce vs repartition on your data
- Review `08_b_Partitions_Concepts.ipynb` to understand partition optimization


## Summary

### What We Learned

1. **What coalesce is**
   - Operation to reduce partitions without full shuffle
   - Combines partitions locally when possible
   - More efficient than repartition when reducing

2. **When to use coalesce**
   - Reducing too many partitions
   - After filtering large datasets
   - Before writing to storage
   - After wide transformations

3. **How coalesce differs from repartition**
   - Coalesce: Minimal shuffle, local combination
   - Repartition: Full shuffle, redistributes all data
   - Coalesce: Can only reduce
   - Repartition: Can increase or decrease

4. **Best practices**
   - Use coalesce when reducing partitions
   - Use repartition when increasing or needing even distribution
   - Always verify partition count
   - Don't coalesce below core count

### The Bottom Line

> **Coalesce is your go-to tool for efficiently reducing partitions. It's faster than repartition because it avoids the expensive full shuffle. But remember: it can only reduce, not increase, and may preserve data skew.**

---

**Related Notebooks:**
- `08_a_Spark_Architecture.ipynb` - Understanding executors, cores, and tasks
- `08_b_Partitions_Concepts.ipynb` - Understanding partitions and optimization


In [8]:
# Clean up
spark.stop()
print("Spark session stopped.")


Spark session stopped.
